In [ ]:
!pip install iterative-stratification

In [1]:
from fastai.vision.all import *
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
# import gc

In [2]:
def random_seed(seed_value, use_cuda): 
    np.random.seed(seed_value)
    torch.manual_seed(seed_value) 
    random.seed(seed_value)
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True 
        torch.backends.cudnn.benchmark = False 

random_seed(3407, True)

In [4]:
train_df = pd.read_csv('/kaggle/input/hackathon-online-find-the-buildings/train.csv')
train_df['file'] = train_df['file'].apply(lambda x : "/kaggle/input/hackathon-online-find-the-buildings/train/train/" + x)

# Prepare submission dataframe
df_pesudo = pd.read_csv('/kaggle/input/eva-buliding/EVA_07-Con-UnAug.csv')

# For pesudo
df_pesudo = df_pesudo[df_pesudo['confidence'] > 0.73]
df_pesudo = df_pesudo[['file', 'label']]
df_pesudo['file'] = df_pesudo['file'].apply(lambda x : "/kaggle/input/hackathon-online-find-the-buildings/test/" + x)

In [5]:
frames = [train_df, df_pesudo]

train_df = pd.concat(frames, ignore_index=True)

train_df

,file,label
0,/kaggle/input/hackathon-online-find-the-buildings/train/train/g_0.png,1
1,/kaggle/input/hackathon-online-find-the-buildings/train/train/g_4.png,1
2,/kaggle/input/hackathon-online-find-the-buildings/train/train/g_6.png,1
3,/kaggle/input/hackathon-online-find-the-buildings/train/train/g_7.png,0
4,/kaggle/input/hackathon-online-find-the-buildings/train/train/g_8.png,1
...,...,...
2064,/kaggle/input/hackathon-online-find-the-buildings/test/g_2254.png,1
2065,/kaggle/input/hackathon-online-find-the-buildings/test/g_2262.png,1
2066,/kaggle/input/hackathon-online-find-the-buildings/test/g_2280.png,1
2067,/kaggle/input/hackathon-online-find-the-buildings/test/g_2282.png,1


In [ ]:
# strat_kfold = MultilabelStratifiedKFold(n_splits=10, random_state=42, shuffle=True)
# train_df['fold'] = -1
# for i, (_, test_index) in enumerate(strat_kfold.split(train_df.file.values, train_df.iloc[:,1:].values)):
#     train_df.iloc[test_index, -1] = i
# train_df.head()

In [ ]:
# train_df.fold.value_counts().plot.bar();

In [6]:
class AlbumentationsTransform (RandTransform):
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [7]:
import albumentations
from albumentations.pytorch import ToTensorV2
warnings.filterwarnings('ignore')

size = 336
def get_train_aug(): return albumentations.Compose([
            albumentations.Resize(always_apply=False, p=1.0, height=size, width=size, interpolation=0),
#             AutoContrast(),
            #albumentations.PadIfNeeded(always_apply=False, p=1.0, min_height=size, min_width=size, pad_height_divisor=None, pad_width_divisor=None, border_mode=0, value=(255, 255, 255), mask_value=None), #Extra tip, use size that's suitable for the efficentNetwork you are using.
            albumentations.Rotate(limit=(-360, 360)),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
    
#             albumentations.Sharpen(always_apply=False, p=1.0, alpha=(0.2, 0.5), lightness=(0.5, 1.0)),
            #albumentations.GaussianBlur(always_apply=False, p=1.0, blur_limit=(3, 7), sigma_limit=(0.0, 0)),
            #albumentations.ShiftScaleRotate(always_apply=False, p=1.0, shift_limit_x=(-0.06, 0.06), shift_limit_y=(-0.06, 0.06), scale_limit=(-0.1, 0.1), rotate_limit=(-20, 20), interpolation=0, border_mode=0, value=(255, 255, 255), mask_value=None, rotate_method='largest_box'),
            # albumentations.HueSaturationValue(
            #     hue_shift_limit=0.2, 
            #     sat_shift_limit=0.2, 
            #     val_shift_limit=0.2, 
            #     p=0.5
            # ),
            # #albumentations.ColorJitter(contrast=(0.9, 1.1)),
#             albumentations.CLAHE(always_apply=False, p=0.5, clip_limit=(1, 4), tile_grid_size=(8, 8)),
#             albumentations.RandomBrightnessContrast(
#                 brightness_limit=(0,0.2), 
#                 contrast_limit=(0, 0.2), 
#                 p=0.5
#             ),
            #albumentations.RandomGamma(),
            #albumentations.ToGray(always_apply=False, p=1.0)
            
           #albumentations.RandomGridShuffle(p=0.5)\
    ], p=1.)



def get_valid_aug(): return albumentations.Compose([
            albumentations.Resize(always_apply=False, p=1.0, height=size, width=size, interpolation=0),
#             albumentations.Sharpen(always_apply=False, p=1.0, alpha=(0.2, 0.5), lightness=(0.5, 1.0))
            #albumentations.PadIfNeeded(always_apply=False, p=1.0, min_height=size, min_width=size, pad_height_divisor=None, pad_width_divisor=None, border_mode=0, value=(255, 255, 255), mask_value=None),
            #albumentations.ToGray(always_apply=False, p=1.0),
            #AutoContrast(),
             
            ], p=1.0)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

In [8]:
def get_data():
    return DataBlock(blocks=(ImageBlock,CategoryBlock),
                       get_x=ColReader(0),
                       get_y=ColReader(1),
                       item_tfms=item_tfms,
                   batch_tfms=[Normalize.from_stats(*imagenet_stats)]
                      ).dataloaders(train_df, bs=32)

In [9]:
test_df = pd.read_csv('/kaggle/input/hackathon-online-find-the-buildings/test.csv')
test_df['file'] = test_df['file'].apply(lambda x : "/kaggle/input/hackathon-online-find-the-buildings/test/" + x)
tstpng = test_df.copy()
tstpng.head()

,file,label
0,/kaggle/input/hackathon-online-find-the-buildings/test/g_1.png,NaN
1,/kaggle/input/hackathon-online-find-the-buildings/test/g_2.png,NaN
2,/kaggle/input/hackathon-online-find-the-buildings/test/g_3.png,NaN
3,/kaggle/input/hackathon-online-find-the-buildings/test/g_5.png,NaN
4,/kaggle/input/hackathon-online-find-the-buildings/test/g_13.png,NaN


In [11]:
dls = get_data()
learn = cnn_learner(dls, 'eva_large_patch14_336.in22k_ft_in1k', metrics=accuracy, cbs=[SaveModelCallback(monitor='valid_loss')]).to_fp16()
learn.model = torch.nn.DataParallel(learn.model)
learn.fine_tune(30)

epoch,train_loss,valid_loss,accuracy,time
0,0.548794,0.292630,0.905569,01:22


Better model found at epoch 0 with valid_loss value: 0.29262983798980713.


epoch,train_loss,valid_loss,accuracy,time
0,0.366766,0.317888,0.917676,01:39
1,0.322936,0.253726,0.924939,01:38
2,0.292496,0.289350,0.905569,01:37
3,0.283601,0.241247,0.927361,01:38
4,0.257288,0.386254,0.886199,01:38


Better model found at epoch 0 with valid_loss value: 0.3178880512714386.
Better model found at epoch 1 with valid_loss value: 0.2537262439727783.
Better model found at epoch 3 with valid_loss value: 0.2412470281124115.


KeyboardInterrupt: 

In [ ]:
import torch.nn.functional as F

tst_dl = learn.dls.test_dl(tstpng)

# get_preds returns probabilities for each class if model's output isn't post-processed
preds, _ = learn.get_preds(dl=tst_dl)

# Apply softmax to obtain probabilities for each class
probabilities = F.softmax(preds, dim=1)

# Calculate confidence as max probability
confidence = probabilities.max(dim=1)[0]

In [ ]:
# Prepare submission dataframe
sub = pd.read_csv('/kaggle/input/hackathon-online-find-the-buildings/test.csv')
idxs = preds.argmax(dim=1)
vocab = np.array(dls.vocab)
sub['label'] = vocab[idxs]

In [ ]:
sub.to_csv('EVA_08_pesudo.csv', index=False)